<a href="https://colab.research.google.com/github/ykitaguchi77/statistics_for_articles/blob/main/GravAI_treatment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#**GravAI Before-After study**

In [ ]:
# prompt: gdrive

from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
import pandas as pd
from scipy.stats import mannwhitneyu

# ファイルパスを設定
file_path = "/content/drive/MyDrive/発表/2025日本眼科AI学会/patientlist'.xlsx"

def run_statistical_analysis(file_path):
    # Excelファイルを読み込む
    print(f"ファイル '{file_path}' を読み込んでいます...")
    df = pd.read_excel(file_path)

    # Step 1: データのクリーニングとフィルタリング
    # 明示的にcopy()を使用してコピーを作成し、SettingWithCopyWarningを回避
    #df_filtered = df[(df['steroid pulse'] != 0) & (df['operation'] == '-')].copy()
    df_filtered = df.copy()

    # Step 2: データのグループ化
    group_negative_30_or_less = df_filtered[df_filtered['difference'] <= -30]
    group_other = df_filtered[df_filtered['difference'] > -30]

    print(f"difference ≤ -30群: {len(group_negative_30_or_less)}件")
    print(f"difference > -30群: {len(group_other)}件")

    # ΔCAS（CASの変化量）を計算 - コピーしたDataFrameに対して操作
    df_filtered['ΔCAS'] = df_filtered['CAS_pre'] - df_filtered['CAS_post']

    # Step 3: ヘルパー関数
    # 目の値（カンマ区切り）を右目と左目の列に分ける関数
    def split_and_convert_eye_values(group, column):
        right_eye, left_eye = [], []
        for value in group[column].dropna():
            if isinstance(value, str):
                parts = value.replace(' ', '').split(',')
                if len(parts) == 2:
                    try:
                        right_eye.append(float(parts[0]))
                        left_eye.append(float(parts[1]))
                    except ValueError:
                        # 数値変換に失敗した場合はスキップ
                        pass
        return pd.Series(right_eye), pd.Series(left_eye)

    # 記述統計を計算する関数
    def descriptive_stats(series):
        series_clean = series.dropna()
        if len(series_clean) == 0:
            return {
                'Min': None,
                'Q1': None,
                'Median': None,
                'Q3': None,
                'Max': None,
                'Mean': None,
                'SD': None,
                'Count': 0
            }
        return {
            'Min': series_clean.min(),
            'Q1': series_clean.quantile(0.25),
            'Median': series_clean.median(),
            'Q3': series_clean.quantile(0.75),
            'Max': series_clean.max(),
            'Mean': series_clean.mean(),
            'SD': series_clean.std(),
            'Count': len(series_clean)
        }

    # Mann-Whitney U検定を実行する関数
    def mann_whitney_test(group1, group2):
        if len(group1.dropna()) > 0 and len(group2.dropna()) > 0:
            stat, p_value = mannwhitneyu(group1.dropna(), group2.dropna(), alternative='two-sided')
            return stat, p_value
        return None, None

    # 眼関連の列に対してΔ変数を計算する関数
    def calculate_delta_for_eye_column(group, pre_col, post_col):
        # 右目と左目のpreとpost値を取得
        right_eye_pre, left_eye_pre = split_and_convert_eye_values(group, pre_col)
        right_eye_post, left_eye_post = split_and_convert_eye_values(group, post_col)

        # 右目のΔ値を計算
        right_eye_delta = pd.Series([])
        if len(right_eye_pre) > 0 and len(right_eye_post) > 0:
            # 各要素ごとにpre/postの対応を取る方法に変更
            # 両方のデータフレームをリセットして新しいインデックスを作成
            right_pre_reset = right_eye_pre.reset_index(drop=True)
            right_post_reset = right_eye_post.reset_index(drop=True)

            # 利用可能なデータの長さを確認
            min_length = min(len(right_pre_reset), len(right_post_reset))
            if min_length > 0:
                # 同じ長さのデータを使用してデルタを計算
                right_eye_delta = right_post_reset.iloc[:min_length] - right_pre_reset.iloc[:min_length]

        # 左目のΔ値を計算
        left_eye_delta = pd.Series([])
        if len(left_eye_pre) > 0 and len(left_eye_post) > 0:
            left_pre_reset = left_eye_pre.reset_index(drop=True)
            left_post_reset = left_eye_post.reset_index(drop=True)

            min_length = min(len(left_pre_reset), len(left_post_reset))
            if min_length > 0:
                left_eye_delta = left_post_reset.iloc[:min_length] - left_pre_reset.iloc[:min_length]

        return right_eye_delta, left_eye_delta

    # Step 4: 分析する変数
    eye_related_columns = [
        'MRD-1_pre', 'MRD1_post',
        'MRD-2_pre', 'MRD2_post',
        'Hertel_pre', 'Hertel_post'
    ]

    non_eye_columns = [
        'Gorman score_pre', 'Gorman score_post',
        'CAS_pre', 'CAS_post', 'ΔCAS'
    ]

    # Step 5: 分析の実行
    results = []

    def append_results(variable, eye, group_name, series, stat, p_value):
        stats = descriptive_stats(series)
        results.append({
            'Variable': variable,
            'Eye': eye,
            'Group': group_name,
            'Statistic': stat,
            'P-value': p_value,
            **stats
        })

    # Δ変数の計算と分析
    delta_columns = []

    # MRD-1のΔ値
    delta_mrd1_name = 'ΔMRD-1'
    delta_columns.append(delta_mrd1_name)
    right_delta_neg_30_mrd1, left_delta_neg_30_mrd1 = calculate_delta_for_eye_column(
        group_negative_30_or_less, 'MRD-1_pre', 'MRD1_post'
    )
    right_delta_other_mrd1, left_delta_other_mrd1 = calculate_delta_for_eye_column(
        group_other, 'MRD-1_pre', 'MRD1_post'
    )

    # MRD-1のΔ値の統計分析
    # 右目Δ
    stat_right_delta_mrd1, p_right_delta_mrd1 = mann_whitney_test(right_delta_neg_30_mrd1, right_delta_other_mrd1)
    append_results(delta_mrd1_name, 'Right', 'difference ≤ -30', right_delta_neg_30_mrd1, stat_right_delta_mrd1, p_right_delta_mrd1)
    append_results(delta_mrd1_name, 'Right', 'difference > -30', right_delta_other_mrd1, stat_right_delta_mrd1, p_right_delta_mrd1)

    # 左目Δ
    stat_left_delta_mrd1, p_left_delta_mrd1 = mann_whitney_test(left_delta_neg_30_mrd1, left_delta_other_mrd1)
    append_results(delta_mrd1_name, 'Left', 'difference ≤ -30', left_delta_neg_30_mrd1, stat_left_delta_mrd1, p_left_delta_mrd1)
    append_results(delta_mrd1_name, 'Left', 'difference > -30', left_delta_other_mrd1, stat_left_delta_mrd1, p_left_delta_mrd1)

    # 両眼Δ
    bilateral_delta_neg_30_mrd1 = pd.concat([right_delta_neg_30_mrd1, left_delta_neg_30_mrd1], ignore_index=True)
    bilateral_delta_other_mrd1 = pd.concat([right_delta_other_mrd1, left_delta_other_mrd1], ignore_index=True)
    stat_bilateral_delta_mrd1, p_bilateral_delta_mrd1 = mann_whitney_test(bilateral_delta_neg_30_mrd1, bilateral_delta_other_mrd1)
    append_results(delta_mrd1_name, 'Bilateral', 'difference ≤ -30', bilateral_delta_neg_30_mrd1, stat_bilateral_delta_mrd1, p_bilateral_delta_mrd1)
    append_results(delta_mrd1_name, 'Bilateral', 'difference > -30', bilateral_delta_other_mrd1, stat_bilateral_delta_mrd1, p_bilateral_delta_mrd1)

    # MRD-2のΔ値
    delta_mrd2_name = 'ΔMRD-2'
    delta_columns.append(delta_mrd2_name)
    right_delta_neg_30_mrd2, left_delta_neg_30_mrd2 = calculate_delta_for_eye_column(
        group_negative_30_or_less, 'MRD-2_pre', 'MRD2_post'
    )
    right_delta_other_mrd2, left_delta_other_mrd2 = calculate_delta_for_eye_column(
        group_other, 'MRD-2_pre', 'MRD2_post'
    )

    # MRD-2のΔ値の統計分析
    # 右目Δ
    stat_right_delta_mrd2, p_right_delta_mrd2 = mann_whitney_test(right_delta_neg_30_mrd2, right_delta_other_mrd2)
    append_results(delta_mrd2_name, 'Right', 'difference ≤ -30', right_delta_neg_30_mrd2, stat_right_delta_mrd2, p_right_delta_mrd2)
    append_results(delta_mrd2_name, 'Right', 'difference > -30', right_delta_other_mrd2, stat_right_delta_mrd2, p_right_delta_mrd2)

    # 左目Δ
    stat_left_delta_mrd2, p_left_delta_mrd2 = mann_whitney_test(left_delta_neg_30_mrd2, left_delta_other_mrd2)
    append_results(delta_mrd2_name, 'Left', 'difference ≤ -30', left_delta_neg_30_mrd2, stat_left_delta_mrd2, p_left_delta_mrd2)
    append_results(delta_mrd2_name, 'Left', 'difference > -30', left_delta_other_mrd2, stat_left_delta_mrd2, p_left_delta_mrd2)

    # 両眼Δ
    bilateral_delta_neg_30_mrd2 = pd.concat([right_delta_neg_30_mrd2, left_delta_neg_30_mrd2], ignore_index=True)
    bilateral_delta_other_mrd2 = pd.concat([right_delta_other_mrd2, left_delta_other_mrd2], ignore_index=True)
    stat_bilateral_delta_mrd2, p_bilateral_delta_mrd2 = mann_whitney_test(bilateral_delta_neg_30_mrd2, bilateral_delta_other_mrd2)
    append_results(delta_mrd2_name, 'Bilateral', 'difference ≤ -30', bilateral_delta_neg_30_mrd2, stat_bilateral_delta_mrd2, p_bilateral_delta_mrd2)
    append_results(delta_mrd2_name, 'Bilateral', 'difference > -30', bilateral_delta_other_mrd2, stat_bilateral_delta_mrd2, p_bilateral_delta_mrd2)

    # Hertelのデルタ値
    delta_hertel_name = 'ΔHertel'
    delta_columns.append(delta_hertel_name)
    right_delta_neg_30_hertel, left_delta_neg_30_hertel = calculate_delta_for_eye_column(
        group_negative_30_or_less, 'Hertel_pre', 'Hertel_post'
    )
    right_delta_other_hertel, left_delta_other_hertel = calculate_delta_for_eye_column(
        group_other, 'Hertel_pre', 'Hertel_post'
    )

    # Hertelのデルタ値の統計分析
    # 右目Δ
    stat_right_delta_hertel, p_right_delta_hertel = mann_whitney_test(right_delta_neg_30_hertel, right_delta_other_hertel)
    append_results(delta_hertel_name, 'Right', 'difference ≤ -30', right_delta_neg_30_hertel, stat_right_delta_hertel, p_right_delta_hertel)
    append_results(delta_hertel_name, 'Right', 'difference > -30', right_delta_other_hertel, stat_right_delta_hertel, p_right_delta_hertel)

    # 左目Δ
    stat_left_delta_hertel, p_left_delta_hertel = mann_whitney_test(left_delta_neg_30_hertel, left_delta_other_hertel)
    append_results(delta_hertel_name, 'Left', 'difference ≤ -30', left_delta_neg_30_hertel, stat_left_delta_hertel, p_left_delta_hertel)
    append_results(delta_hertel_name, 'Left', 'difference > -30', left_delta_other_hertel, stat_left_delta_hertel, p_left_delta_hertel)

    # 両眼Δ
    bilateral_delta_neg_30_hertel = pd.concat([right_delta_neg_30_hertel, left_delta_neg_30_hertel], ignore_index=True)
    bilateral_delta_other_hertel = pd.concat([right_delta_other_hertel, left_delta_other_hertel], ignore_index=True)
    stat_bilateral_delta_hertel, p_bilateral_delta_hertel = mann_whitney_test(bilateral_delta_neg_30_hertel, bilateral_delta_other_hertel)
    append_results(delta_hertel_name, 'Bilateral', 'difference ≤ -30', bilateral_delta_neg_30_hertel, stat_bilateral_delta_hertel, p_bilateral_delta_hertel)
    append_results(delta_hertel_name, 'Bilateral', 'difference > -30', bilateral_delta_other_hertel, stat_bilateral_delta_hertel, p_bilateral_delta_hertel)

    # Gorman scoreのΔ値を計算
    delta_gorman_col = 'ΔGorman score'
    delta_columns.append(delta_gorman_col)
    try:
        group_neg_30_gorman_pre = pd.to_numeric(group_negative_30_or_less['Gorman score_pre'], errors='coerce')
        group_neg_30_gorman_post = pd.to_numeric(group_negative_30_or_less['Gorman score_post'], errors='coerce')
        group_neg_30_delta_gorman = group_neg_30_gorman_post - group_neg_30_gorman_pre

        group_other_gorman_pre = pd.to_numeric(group_other['Gorman score_pre'], errors='coerce')
        group_other_gorman_post = pd.to_numeric(group_other['Gorman score_post'], errors='coerce')
        group_other_delta_gorman = group_other_gorman_post - group_other_gorman_pre

        # Mann-Whitney U検定
        stat_gorman, p_gorman = mann_whitney_test(group_neg_30_delta_gorman, group_other_delta_gorman)

        # 結果を追加
        append_results(delta_gorman_col, 'Both', 'difference ≤ -30', group_neg_30_delta_gorman, stat_gorman, p_gorman)
        append_results(delta_gorman_col, 'Both', 'difference > -30', group_other_delta_gorman, stat_gorman, p_gorman)
    except KeyError:
        print(f"警告: 列 'Gorman score_pre' または 'Gorman score_post' はデータフレームに存在しません。スキップします。")

    # 元の変数も分析
    # 眼関連の列を分析
    for column in eye_related_columns:
        # 右目
        right_eye_neg_30, _ = split_and_convert_eye_values(group_negative_30_or_less, column)
        right_eye_other, _ = split_and_convert_eye_values(group_other, column)
        stat_right, p_right = mann_whitney_test(right_eye_neg_30, right_eye_other)
        append_results(column, 'Right', 'difference ≤ -30', right_eye_neg_30, stat_right, p_right)
        append_results(column, 'Right', 'difference > -30', right_eye_other, stat_right, p_right)

        # 左目
        _, left_eye_neg_30 = split_and_convert_eye_values(group_negative_30_or_less, column)
        _, left_eye_other = split_and_convert_eye_values(group_other, column)
        stat_left, p_left = mann_whitney_test(left_eye_neg_30, left_eye_other)
        append_results(column, 'Left', 'difference ≤ -30', left_eye_neg_30, stat_left, p_left)
        append_results(column, 'Left', 'difference > -30', left_eye_other, stat_left, p_left)

        # 両眼（右目と左目のデータを結合）
        bilateral_neg_30 = pd.concat([right_eye_neg_30, left_eye_neg_30], ignore_index=True)
        bilateral_other = pd.concat([right_eye_other, left_eye_other], ignore_index=True)
        stat_bilateral, p_bilateral = mann_whitney_test(bilateral_neg_30, bilateral_other)
        append_results(column, 'Bilateral', 'difference ≤ -30', bilateral_neg_30, stat_bilateral, p_bilateral)
        append_results(column, 'Bilateral', 'difference > -30', bilateral_other, stat_bilateral, p_bilateral)

    # 非眼関連の列を分析
    for column in non_eye_columns:
        try:
            group1_series = pd.to_numeric(group_negative_30_or_less[column], errors='coerce').dropna()
            group2_series = pd.to_numeric(group_other[column], errors='coerce').dropna()
            stat, p_value = mann_whitney_test(group1_series, group2_series)
            append_results(column, 'Both', 'difference ≤ -30', group1_series, stat, p_value)
            append_results(column, 'Both', 'difference > -30', group2_series, stat, p_value)
        except KeyError:
            print(f"警告: 列 '{column}' はデータフレームに存在しません。スキップします。")

    # 結果をDataFrameに変換
    results_df = pd.DataFrame(results)

    # p値の閾値に基づいて有意性を示すDataFrameを作成
    significance_threshold = 0.05

    # 有意性の列を生成
    significance_results = results_df.copy()
    significance_results['Significance'] = significance_results['P-value'].apply(
        lambda p: 'Significant' if p is not None and p < significance_threshold else 'Not Significant')

    # デフォルト出力では小数点以下が多すぎるため、数値の表示形式を整える
    for col in ['Statistic', 'P-value', 'Min', 'Q1', 'Median', 'Q3', 'Max', 'Mean', 'SD']:
        if col in significance_results.columns:
            significance_results[col] = significance_results[col].apply(
                lambda x: f"{x:.4f}" if isinstance(x, (int, float)) else x)

    # 結果を表示
    print("\n===== 統計分析結果 =====")
    print(significance_results)

    # Mean±SDの形式で結果を表示
    print("\n===== Mean±SD形式の結果 =====")

    # 各測定項目のグループ定義
    measurement_groups = {
        'MRD-1': ['MRD-1_pre', 'MRD1_post', 'ΔMRD-1'],
        'MRD-2': ['MRD-2_pre', 'MRD2_post', 'ΔMRD-2'],
        'Hertel': ['Hertel_pre', 'Hertel_post', 'ΔHertel'],
        'Gorman score': ['Gorman score_pre', 'Gorman score_post', 'ΔGorman score'],
        'CAS': ['CAS_pre', 'CAS_post', 'ΔCAS']
    }

    # 利用可能な変数のセット
    available_variables = set(significance_results['Variable'].unique())

    # 各グループについて結果を表示
    for group_name, variables in measurement_groups.items():
        # このグループに存在する変数をフィルタリング
        group_variables = [var for var in variables if var in available_variables]

        if not group_variables:
            continue  # このグループの変数が一つもない場合はスキップ

        print(f"\n----- {group_name}グループ -----")

        # 表示する目のタイプを決定
        if any(var in eye_related_columns for var in group_variables) or \
           any(var.startswith('Δ') and var not in ['ΔCAS', 'ΔGorman score'] for var in group_variables):
            eye_types = ['Right', 'Left', 'Bilateral']
        else:
            eye_types = ['Both']

        # 各目のタイプについて表示
        for eye_type in eye_types:
            has_data_for_eye = False
            eye_results = []

            # この目のタイプでの各変数の結果を収集
            for variable in group_variables:
                # 目のタイプによるフィルタリング
                if eye_type in ['Right', 'Left', 'Bilateral'] and variable in non_eye_columns:
                    continue
                if eye_type == 'Both' and variable in eye_related_columns:
                    continue
                if eye_type in ['Right', 'Left', 'Bilateral'] and variable.startswith('Δ') and variable in ['ΔCAS', 'ΔGorman score']:
                    continue
                if eye_type == 'Both' and variable.startswith('Δ') and variable not in ['ΔCAS', 'ΔGorman score']:
                    continue

                # データを取得
                rows = significance_results[
                    (significance_results['Variable'] == variable) &
                    (significance_results['Eye'] == eye_type)
                ]

                if len(rows) >= 2:  # 両グループのデータがある場合
                    has_data_for_eye = True
                    neg_30_row = rows[rows['Group'] == 'difference ≤ -30'].iloc[0]
                    other_row = rows[rows['Group'] == 'difference > -30'].iloc[0]

                    # 値を取得して数値に変換
                    mean_neg_30 = float(neg_30_row['Mean']) if isinstance(neg_30_row['Mean'], str) else neg_30_row['Mean']
                    sd_neg_30 = float(neg_30_row['SD']) if isinstance(neg_30_row['SD'], str) else neg_30_row['SD']
                    mean_other = float(other_row['Mean']) if isinstance(other_row['Mean'], str) else other_row['Mean']
                    sd_other = float(other_row['SD']) if isinstance(other_row['SD'], str) else other_row['SD']
                    p_value = float(neg_30_row['P-value']) if isinstance(neg_30_row['P-value'], str) else neg_30_row['P-value']

                    # 結果を保存
                    if mean_neg_30 is not None and sd_neg_30 is not None and mean_other is not None and sd_other is not None:
                        eye_results.append({
                            'variable': variable,
                            'mean_neg_30': mean_neg_30,
                            'sd_neg_30': sd_neg_30,
                            'count_neg_30': neg_30_row['Count'],
                            'mean_other': mean_other,
                            'sd_other': sd_other,
                            'count_other': other_row['Count'],
                            'p_value': p_value
                        })

            # この目のタイプにデータがある場合、表示
            if has_data_for_eye:
                print(f"\n{group_name} ({eye_type}):")

                for result in eye_results:
                    print(f"  {result['variable']}:")
                    print(f"    ≤ -30群: {result['mean_neg_30']:.2f}±{result['sd_neg_30']:.2f} (n={result['count_neg_30']})")
                    print(f"    > -30群: {result['mean_other']:.2f}±{result['sd_other']:.2f} (n={result['count_other']})")
                    print(f"    p値: {result['p_value']:.4f}")
                    if result['p_value'] < 0.05:
                        print("    有意差: あり")
                    else:
                        print("    有意差: なし")

    # Excelに保存
    output_filename = 'statistical_analysis_results.xlsx'
    significance_results.to_excel(output_filename, index=False)
    print(f"\nExcelファイル '{output_filename}' に結果を保存しました。")

    # 結果からp値が0.05未満の変数を抽出（有意差のある項目）
    significant_results = significance_results[
        (significance_results['P-value'].apply(
            lambda p: False if p is None else float(p) < significance_threshold
        )) &
        (significance_results['Group'] == 'difference ≤ -30')
    ]

    if len(significant_results) > 0:
        print("\n===== 有意差のある項目 =====")
        significant_variables = significant_results[['Variable', 'Eye', 'P-value']]
        print(significant_variables)
    else:
        print("\n有意差のある項目はありませんでした。")

    return significance_results



# 統計分析を実行するコード
# ファイルパスを設定（自分の環境に合わせて変更してください）
file_path = "/content/drive/MyDrive/発表/2025日本眼科AI学会/patientlist'.xlsx"

# 関数を実行
results = run_statistical_analysis(file_path)

ファイル '/content/drive/MyDrive/発表/2025日本眼科AI学会/patientlist'.xlsx' を読み込んでいます...
difference ≤ -30群: 12件
difference > -30群: 30件
警告: 列 'ΔCAS' はデータフレームに存在しません。スキップします。

===== 統計分析結果 =====
             Variable        Eye             Group Statistic P-value      Min  \
0              ΔMRD-1      Right  difference ≤ -30   64.0000  0.0009  -6.5000   
1              ΔMRD-1      Right  difference > -30   64.0000  0.0009  -4.0000   
2              ΔMRD-1       Left  difference ≤ -30  175.0000  0.8953  -2.0000   
3              ΔMRD-1       Left  difference > -30  175.0000  0.8953  -3.0000   
4              ΔMRD-1  Bilateral  difference ≤ -30  465.0000  0.0089  -6.5000   
..                ...        ...               ...       ...     ...      ...   
59  Gorman score_post       Both  difference > -30  107.5000  0.4664   1.0000   
60            CAS_pre       Both  difference ≤ -30  151.0000  0.3997   0.0000   
61            CAS_pre       Both  difference > -30  151.0000  0.3997   0.0000   
62       

In [ ]:
import pandas as pd
from scipy.stats import mannwhitneyu

# pandas表示オプションを設定（結果を省略なしで表示するため）
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', None)

# ファイルパスを設定
file_path = "/content/drive/MyDrive/発表/2025日本眼科AI学会/patientlist'.xlsx"

def run_statistical_analysis(file_path):
    # Excelファイルを読み込む
    print(f"ファイル '{file_path}' を読み込んでいます...")
    df = pd.read_excel(file_path)

    # Step 1: データのクリーニングとフィルタリング
    # 明示的にcopy()を使用してコピーを作成し、SettingWithCopyWarningを回避
    df_filtered = df[(df['steroid pulse'] != 0) & (df['operation'] == '-')].copy()

    # Step 2: データのグループ化
    group_negative_30_or_less = df_filtered[df_filtered['difference'] <= -30]
    group_other = df_filtered[df_filtered['difference'] > -30]

    print(f"difference ≤ -30群: {len(group_negative_30_or_less)}件")
    print(f"difference > -30群: {len(group_other)}件")

    # ΔCAS（CASの変化量）を計算 - コピーしたDataFrameに対して操作
    df_filtered['ΔCAS'] = df_filtered['CAS_pre'] - df_filtered['CAS_post']

    # Step 3: ヘルパー関数
    # 目の値（カンマ区切り）を右目と左目の列に分ける関数
    def split_and_convert_eye_values(group, column):
        right_eye, left_eye = [], []
        for value in group[column].dropna():
            if isinstance(value, str):
                parts = value.replace(' ', '').split(',')
                if len(parts) == 2:
                    try:
                        right_eye.append(float(parts[0]))
                        left_eye.append(float(parts[1]))
                    except ValueError:
                        # 数値変換に失敗した場合はスキップ
                        pass
        return pd.Series(right_eye), pd.Series(left_eye)

    # 記述統計を計算する関数
    def descriptive_stats(series):
        series_clean = series.dropna()
        if len(series_clean) == 0:
            return {
                'Min': None,
                'Q1': None,
                'Median': None,
                'Q3': None,
                'Max': None,
                'Mean': None,
                'SD': None,
                'Count': 0
            }
        return {
            'Min': series_clean.min(),
            'Q1': series_clean.quantile(0.25),
            'Median': series_clean.median(),
            'Q3': series_clean.quantile(0.75),
            'Max': series_clean.max(),
            'Mean': series_clean.mean(),
            'SD': series_clean.std(),
            'Count': len(series_clean)
        }

    # Mann-Whitney U検定を実行する関数
    def mann_whitney_test(group1, group2):
        if len(group1.dropna()) > 0 and len(group2.dropna()) > 0:
            stat, p_value = mannwhitneyu(group1.dropna(), group2.dropna(), alternative='two-sided')
            return stat, p_value
        return None, None

    # 眼関連の列に対してΔ変数を計算する関数
    def calculate_delta_for_eye_column(group, pre_col, post_col):
        # 右目と左目のpreとpost値を取得
        right_eye_pre, left_eye_pre = split_and_convert_eye_values(group, pre_col)
        right_eye_post, left_eye_post = split_and_convert_eye_values(group, post_col)

        # Δ値を計算（post - pre）
        if len(right_eye_pre) == len(right_eye_post) and len(left_eye_pre) == len(left_eye_post):
            right_eye_delta = right_eye_post.reset_index(drop=True) - right_eye_pre.reset_index(drop=True)
            left_eye_delta = left_eye_post.reset_index(drop=True) - left_eye_pre.reset_index(drop=True)
        else:
            # 長さが一致しない場合は空のSeriesを返す
            right_eye_delta = pd.Series([])
            left_eye_delta = pd.Series([])

        return right_eye_delta, left_eye_delta

    # Step 4: 分析する変数
    eye_related_columns = [
        'MRD-1_pre', 'MRD1_post',
        'MRD-2_pre', 'MRD2_post',
        'Hertel_pre', 'Hertel_post'
    ]

    non_eye_columns = [
        'Gorman score_pre', 'Gorman score_post',
        'CAS_pre', 'CAS_post', 'ΔCAS'
    ]

    # Step 5: 分析の実行
    results = []

    def append_results(variable, eye, group_name, series, stat, p_value):
        stats = descriptive_stats(series)
        results.append({
            'Variable': variable,
            'Eye': eye,
            'Group': group_name,
            'Statistic': stat,
            'P-value': p_value,
            **stats
        })

    # Δ変数の計算と分析
    delta_columns = []

    # MRD-1のΔ値
    delta_mrd1_name = 'ΔMRD-1'
    delta_columns.append(delta_mrd1_name)
    right_delta_neg_30_mrd1, left_delta_neg_30_mrd1 = calculate_delta_for_eye_column(
        group_negative_30_or_less, 'MRD-1_pre', 'MRD1_post'
    )
    right_delta_other_mrd1, left_delta_other_mrd1 = calculate_delta_for_eye_column(
        group_other, 'MRD-1_pre', 'MRD1_post'
    )

    # MRD-1のΔ値の統計分析
    # 右目Δ
    stat_right_delta_mrd1, p_right_delta_mrd1 = mann_whitney_test(right_delta_neg_30_mrd1, right_delta_other_mrd1)
    append_results(delta_mrd1_name, 'Right', 'difference ≤ -30', right_delta_neg_30_mrd1, stat_right_delta_mrd1, p_right_delta_mrd1)
    append_results(delta_mrd1_name, 'Right', 'difference > -30', right_delta_other_mrd1, stat_right_delta_mrd1, p_right_delta_mrd1)

    # 左目Δ
    stat_left_delta_mrd1, p_left_delta_mrd1 = mann_whitney_test(left_delta_neg_30_mrd1, left_delta_other_mrd1)
    append_results(delta_mrd1_name, 'Left', 'difference ≤ -30', left_delta_neg_30_mrd1, stat_left_delta_mrd1, p_left_delta_mrd1)
    append_results(delta_mrd1_name, 'Left', 'difference > -30', left_delta_other_mrd1, stat_left_delta_mrd1, p_left_delta_mrd1)

    # 両眼Δ
    bilateral_delta_neg_30_mrd1 = pd.concat([right_delta_neg_30_mrd1, left_delta_neg_30_mrd1], ignore_index=True)
    bilateral_delta_other_mrd1 = pd.concat([right_delta_other_mrd1, left_delta_other_mrd1], ignore_index=True)
    stat_bilateral_delta_mrd1, p_bilateral_delta_mrd1 = mann_whitney_test(bilateral_delta_neg_30_mrd1, bilateral_delta_other_mrd1)
    append_results(delta_mrd1_name, 'Bilateral', 'difference ≤ -30', bilateral_delta_neg_30_mrd1, stat_bilateral_delta_mrd1, p_bilateral_delta_mrd1)
    append_results(delta_mrd1_name, 'Bilateral', 'difference > -30', bilateral_delta_other_mrd1, stat_bilateral_delta_mrd1, p_bilateral_delta_mrd1)

    # MRD-2のΔ値
    delta_mrd2_name = 'ΔMRD-2'
    delta_columns.append(delta_mrd2_name)
    right_delta_neg_30_mrd2, left_delta_neg_30_mrd2 = calculate_delta_for_eye_column(
        group_negative_30_or_less, 'MRD-2_pre', 'MRD2_post'
    )
    right_delta_other_mrd2, left_delta_other_mrd2 = calculate_delta_for_eye_column(
        group_other, 'MRD-2_pre', 'MRD2_post'
    )

    # MRD-2のΔ値の統計分析
    # 右目Δ
    stat_right_delta_mrd2, p_right_delta_mrd2 = mann_whitney_test(right_delta_neg_30_mrd2, right_delta_other_mrd2)
    append_results(delta_mrd2_name, 'Right', 'difference ≤ -30', right_delta_neg_30_mrd2, stat_right_delta_mrd2, p_right_delta_mrd2)
    append_results(delta_mrd2_name, 'Right', 'difference > -30', right_delta_other_mrd2, stat_right_delta_mrd2, p_right_delta_mrd2)

    # 左目Δ
    stat_left_delta_mrd2, p_left_delta_mrd2 = mann_whitney_test(left_delta_neg_30_mrd2, left_delta_other_mrd2)
    append_results(delta_mrd2_name, 'Left', 'difference ≤ -30', left_delta_neg_30_mrd2, stat_left_delta_mrd2, p_left_delta_mrd2)
    append_results(delta_mrd2_name, 'Left', 'difference > -30', left_delta_other_mrd2, stat_left_delta_mrd2, p_left_delta_mrd2)

    # 両眼Δ
    bilateral_delta_neg_30_mrd2 = pd.concat([right_delta_neg_30_mrd2, left_delta_neg_30_mrd2], ignore_index=True)
    bilateral_delta_other_mrd2 = pd.concat([right_delta_other_mrd2, left_delta_other_mrd2], ignore_index=True)
    stat_bilateral_delta_mrd2, p_bilateral_delta_mrd2 = mann_whitney_test(bilateral_delta_neg_30_mrd2, bilateral_delta_other_mrd2)
    append_results(delta_mrd2_name, 'Bilateral', 'difference ≤ -30', bilateral_delta_neg_30_mrd2, stat_bilateral_delta_mrd2, p_bilateral_delta_mrd2)
    append_results(delta_mrd2_name, 'Bilateral', 'difference > -30', bilateral_delta_other_mrd2, stat_bilateral_delta_mrd2, p_bilateral_delta_mrd2)

    # Hertelのデルタ値（ΔHertel = Hertel_post - Hertel_pre）
    delta_hertel_name = 'ΔHertel'
    delta_columns.append(delta_hertel_name)
    right_delta_neg_30_hertel, left_delta_neg_30_hertel = calculate_delta_for_eye_column(
        group_negative_30_or_less, 'Hertel_pre', 'Hertel_post'
    )
    right_delta_other_hertel, left_delta_other_hertel = calculate_delta_for_eye_column(
        group_other, 'Hertel_pre', 'Hertel_post'
    )

    # Hertelのデルタ値の統計分析
    # 右目Δ
    stat_right_delta_hertel, p_right_delta_hertel = mann_whitney_test(right_delta_neg_30_hertel, right_delta_other_hertel)
    append_results(delta_hertel_name, 'Right', 'difference ≤ -30', right_delta_neg_30_hertel, stat_right_delta_hertel, p_right_delta_hertel)
    append_results(delta_hertel_name, 'Right', 'difference > -30', right_delta_other_hertel, stat_right_delta_hertel, p_right_delta_hertel)

    # 左目Δ
    stat_left_delta_hertel, p_left_delta_hertel = mann_whitney_test(left_delta_neg_30_hertel, left_delta_other_hertel)
    append_results(delta_hertel_name, 'Left', 'difference ≤ -30', left_delta_neg_30_hertel, stat_left_delta_hertel, p_left_delta_hertel)
    append_results(delta_hertel_name, 'Left', 'difference > -30', left_delta_other_hertel, stat_left_delta_hertel, p_left_delta_hertel)

    # 両眼Δ
    bilateral_delta_neg_30_hertel = pd.concat([right_delta_neg_30_hertel, left_delta_neg_30_hertel], ignore_index=True)
    bilateral_delta_other_hertel = pd.concat([right_delta_other_hertel, left_delta_other_hertel], ignore_index=True)
    stat_bilateral_delta_hertel, p_bilateral_delta_hertel = mann_whitney_test(bilateral_delta_neg_30_hertel, bilateral_delta_other_hertel)
    append_results(delta_hertel_name, 'Bilateral', 'difference ≤ -30', bilateral_delta_neg_30_hertel, stat_bilateral_delta_hertel, p_bilateral_delta_hertel)
    append_results(delta_hertel_name, 'Bilateral', 'difference > -30', bilateral_delta_other_hertel, stat_bilateral_delta_hertel, p_bilateral_delta_hertel)

    # Gorman scoreのΔ値を計算
    delta_gorman_col = 'ΔGorman score'
    delta_columns.append(delta_gorman_col)
    try:
        group_neg_30_gorman_pre = pd.to_numeric(group_negative_30_or_less['Gorman score_pre'], errors='coerce')
        group_neg_30_gorman_post = pd.to_numeric(group_negative_30_or_less['Gorman score_post'], errors='coerce')
        group_neg_30_delta_gorman = group_neg_30_gorman_post - group_neg_30_gorman_pre

        group_other_gorman_pre = pd.to_numeric(group_other['Gorman score_pre'], errors='coerce')
        group_other_gorman_post = pd.to_numeric(group_other['Gorman score_post'], errors='coerce')
        group_other_delta_gorman = group_other_gorman_post - group_other_gorman_pre

        # Mann-Whitney U検定
        stat_gorman, p_gorman = mann_whitney_test(group_neg_30_delta_gorman, group_other_delta_gorman)

        # 結果を追加
        append_results(delta_gorman_col, 'Both', 'difference ≤ -30', group_neg_30_delta_gorman, stat_gorman, p_gorman)
        append_results(delta_gorman_col, 'Both', 'difference > -30', group_other_delta_gorman, stat_gorman, p_gorman)
    except KeyError:
        print(f"警告: 列 'Gorman score_pre' または 'Gorman score_post' はデータフレームに存在しません。スキップします。")

    # 元の変数も分析
    # 眼関連の列を分析
    for column in eye_related_columns:
        # 右目
        right_eye_neg_30, _ = split_and_convert_eye_values(group_negative_30_or_less, column)
        right_eye_other, _ = split_and_convert_eye_values(group_other, column)
        stat_right, p_right = mann_whitney_test(right_eye_neg_30, right_eye_other)
        append_results(column, 'Right', 'difference ≤ -30', right_eye_neg_30, stat_right, p_right)
        append_results(column, 'Right', 'difference > -30', right_eye_other, stat_right, p_right)

        # 左目
        _, left_eye_neg_30 = split_and_convert_eye_values(group_negative_30_or_less, column)
        _, left_eye_other = split_and_convert_eye_values(group_other, column)
        stat_left, p_left = mann_whitney_test(left_eye_neg_30, left_eye_other)
        append_results(column, 'Left', 'difference ≤ -30', left_eye_neg_30, stat_left, p_left)
        append_results(column, 'Left', 'difference > -30', left_eye_other, stat_left, p_left)

        # 両眼（右目と左目のデータを結合）
        bilateral_neg_30 = pd.concat([right_eye_neg_30, left_eye_neg_30], ignore_index=True)
        bilateral_other = pd.concat([right_eye_other, left_eye_other], ignore_index=True)
        stat_bilateral, p_bilateral = mann_whitney_test(bilateral_neg_30, bilateral_other)
        append_results(column, 'Bilateral', 'difference ≤ -30', bilateral_neg_30, stat_bilateral, p_bilateral)
        append_results(column, 'Bilateral', 'difference > -30', bilateral_other, stat_bilateral, p_bilateral)

    # 非眼関連の列を分析
    for column in non_eye_columns:
        try:
            group1_series = pd.to_numeric(group_negative_30_or_less[column], errors='coerce').dropna()
            group2_series = pd.to_numeric(group_other[column], errors='coerce').dropna()
            stat, p_value = mann_whitney_test(group1_series, group2_series)
            append_results(column, 'Both', 'difference ≤ -30', group1_series, stat, p_value)
            append_results(column, 'Both', 'difference > -30', group2_series, stat, p_value)
        except KeyError:
            print(f"警告: 列 '{column}' はデータフレームに存在しません。スキップします。")

    # 結果をDataFrameに変換
    results_df = pd.DataFrame(results)

    # p値の閾値に基づいて有意性を示すDataFrameを作成
    significance_threshold = 0.05

    # 有意性の列を生成
    significance_results = results_df.copy()
    significance_results['Significance'] = significance_results['P-value'].apply(
        lambda p: 'Significant' if p is not None and p < significance_threshold else 'Not Significant')

    # デフォルト出力では小数点以下が多すぎるため、数値の表示形式を整える
    for col in ['Statistic', 'P-value', 'Min', 'Q1', 'Median', 'Q3', 'Max', 'Mean', 'SD']:
        if col in significance_results.columns:
            significance_results[col] = significance_results[col].apply(
                lambda x: f"{x:.4f}" if isinstance(x, (int, float)) else x)

    # 結果を表示（省略なしで完全表示）
    print("\n===== 統計分析結果（完全表示） =====")
    print("=" * 120)

    # DataFrameを完全に表示するため、to_string()を使用
    print(significance_results.to_string(index=False))

    print("=" * 120)

    # 詳細な表示（変数ごとに分けて表示）
    print("\n===== 変数別詳細結果 =====")
    unique_variables = significance_results['Variable'].unique()

    for i, variable in enumerate(unique_variables):
        print(f"\n--- {i+1}. {variable} ---")
        var_results = significance_results[significance_results['Variable'] == variable]
        print(var_results.to_string(index=False))
        print("-" * 60)

    # Mean±SDの形式で結果を表示
    print("\n===== Mean±SD形式の結果 =====")

    # 各測定項目のグループ定義
    measurement_groups = {
        'MRD-1': ['MRD-1_pre', 'MRD1_post', 'ΔMRD-1'],
        'MRD-2': ['MRD-2_pre', 'MRD2_post', 'ΔMRD-2'],
        'Hertel': ['Hertel_pre', 'Hertel_post', 'ΔHertel'],
        'Gorman score': ['Gorman score_pre', 'Gorman score_post', 'ΔGorman score'],
        'CAS': ['CAS_pre', 'CAS_post', 'ΔCAS']
    }

    # 利用可能な変数のセット
    available_variables = set(significance_results['Variable'].unique())

    # 各グループについて結果を表示
    for group_name, variables in measurement_groups.items():
        # このグループに存在する変数をフィルタリング
        group_variables = [var for var in variables if var in available_variables]
        if not group_variables:
            continue  # このグループの変数が一つもない場合はスキップ

        print(f"\n----- {group_name}グループ -----")

        # 表示する目のタイプを決定
        if any(var in eye_related_columns for var in group_variables) or \
           any(var.startswith('Δ') and var not in ['ΔCAS', 'ΔGorman score'] for var in group_variables):
            eye_types = ['Right', 'Left', 'Bilateral']
        else:
            eye_types = ['Both']

        # 各目のタイプについて表示
        for eye_type in eye_types:
            has_data_for_eye = False
            eye_results = []

            # この目のタイプでの各変数の結果を収集
            for variable in group_variables:
                # 目のタイプによるフィルタリング
                if eye_type in ['Right', 'Left', 'Bilateral'] and variable in non_eye_columns:
                    continue
                if eye_type == 'Both' and variable in eye_related_columns:
                    continue
                if eye_type in ['Right', 'Left', 'Bilateral'] and variable.startswith('Δ') and variable in ['ΔCAS', 'ΔGorman score']:
                    continue
                if eye_type == 'Both' and variable.startswith('Δ') and variable not in ['ΔCAS', 'ΔGorman score']:
                    continue

                # データを取得
                rows = significance_results[
                    (significance_results['Variable'] == variable) &
                    (significance_results['Eye'] == eye_type)
                ]

                if len(rows) >= 2:  # 両グループのデータがある場合
                    has_data_for_eye = True
                    neg_30_row = rows[rows['Group'] == 'difference ≤ -30'].iloc[0]
                    other_row = rows[rows['Group'] == 'difference > -30'].iloc[0]

                    # 値を取得して数値に変換
                    mean_neg_30 = float(neg_30_row['Mean']) if isinstance(neg_30_row['Mean'], str) and neg_30_row['Mean'] != 'None' else neg_30_row['Mean']
                    sd_neg_30 = float(neg_30_row['SD']) if isinstance(neg_30_row['SD'], str) and neg_30_row['SD'] != 'None' else neg_30_row['SD']
                    mean_other = float(other_row['Mean']) if isinstance(other_row['Mean'], str) and other_row['Mean'] != 'None' else other_row['Mean']
                    sd_other = float(other_row['SD']) if isinstance(other_row['SD'], str) and other_row['SD'] != 'None' else other_row['SD']
                    p_value = float(neg_30_row['P-value']) if isinstance(neg_30_row['P-value'], str) and neg_30_row['P-value'] != 'None' else neg_30_row['P-value']

                    # 結果を保存 - 元のコードと同様に欠損値がある場合はスキップ
                    if mean_neg_30 is not None and sd_neg_30 is not None and mean_other is not None and sd_other is not None:
                        # NaNかどうかの追加チェック
                        if not (pd.isna(mean_neg_30) or pd.isna(sd_neg_30) or pd.isna(mean_other) or pd.isna(sd_other)):
                            eye_results.append({
                                'variable': variable,
                                'mean_neg_30': mean_neg_30,
                                'sd_neg_30': sd_neg_30,
                                'count_neg_30': neg_30_row['Count'],
                                'mean_other': mean_other,
                                'sd_other': sd_other,
                                'count_other': other_row['Count'],
                                'p_value': p_value
                            })

            # この目のタイプにデータがある場合、表示
            if has_data_for_eye:
                print(f"\n{group_name} ({eye_type}):")
                for result in eye_results:
                    print(f"  {result['variable']}:")
                    print(f"    ≤ -30群: {result['mean_neg_30']:.2f}±{result['sd_neg_30']:.2f} (n={result['count_neg_30']})")
                    print(f"    > -30群: {result['mean_other']:.2f}±{result['sd_other']:.2f} (n={result['count_other']})")
                    # p値の表示（元のコードに近づける）
                    print(f"    p値: {result['p_value']:.4f}")
                    if result['p_value'] < 0.05:
                        print("    有意差: あり")
                    else:
                        print("    有意差: なし")

    # Excelに保存
    output_filename = 'statistical_analysis_results.xlsx'
    significance_results.to_excel(output_filename, index=False)
    print(f"\nExcelファイル '{output_filename}' に結果を保存しました。")

    # 結果からp値が0.05未満の変数を抽出（有意差のある項目）
    significant_results = significance_results[
        (significance_results['P-value'].apply(
            lambda p: False if p is None else float(p) < significance_threshold
        )) &
        (significance_results['Group'] == 'difference ≤ -30')
    ]

    if len(significant_results) > 0:
        print("\n===== 有意差のある項目（完全表示） =====")
        print("=" * 80)
        significant_variables = significant_results[['Variable', 'Eye', 'P-value']]
        print(significant_variables.to_string(index=False))
        print("=" * 80)
    else:
        print("\n有意差のある項目はありませんでした。")

    return significance_results

# 統計分析を実行するコード
# ファイルパスを設定（自分の環境に合わせて変更してください）
file_path = "/content/drive/MyDrive/発表/2025日本眼科AI学会/patientlist'.xlsx"

# 関数を実行
results = run_statistical_analysis(file_path)

ファイル '/content/drive/MyDrive/発表/2025日本眼科AI学会/patientlist'.xlsx' を読み込んでいます...
difference ≤ -30群: 6件
difference > -30群: 14件
警告: 列 'ΔCAS' はデータフレームに存在しません。スキップします。

===== 統計分析結果（完全表示） =====
         Variable       Eye            Group Statistic P-value     Min      Q1  Median      Q3     Max    Mean     SD  Count    Significance
           ΔMRD-1     Right difference ≤ -30   28.5000  0.2685 -2.0000 -1.3750 -1.0000 -0.2500  0.0000 -0.9167 0.8010      6 Not Significant
           ΔMRD-1     Right difference > -30   28.5000  0.2685 -4.0000 -0.5000 -0.5000  0.0000  1.0000 -0.5714 1.1411     14 Not Significant
           ΔMRD-1      Left difference ≤ -30   43.5000  0.9318 -2.0000 -0.8750 -0.2500  0.0000  0.0000 -0.5833 0.8010      6 Not Significant
           ΔMRD-1      Left difference > -30   43.5000  0.9318 -3.0000 -0.8750 -0.5000  0.0000  2.0000 -0.5000 1.0742     14 Not Significant
           ΔMRD-1 Bilateral difference ≤ -30  143.0000  0.4541 -2.0000 -1.1250 -0.7500  0.0000  0.0000 -0.750

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import matplotlib as mpl
import os

# Google Colabでの日本語フォント対応（最初に実行する必要があります）
# 実行時に以下のコマンドをセルで実行してフォントをインストール
# !apt-get -y install fonts-ipafont-gothic fonts-ipafont-mincho

# フォント設定（Google Colab環境用に最適化）
def setup_japanese_fonts():
    # Colab環境でのフォントインストール確認
    if not os.path.exists('/usr/share/fonts/truetype/fonts-japanese-gothic.ttf'):
        print("警告: 日本語フォントが見つかりません。")
        print("以下のコマンドを別セルで実行してください:")
        print("!apt-get -y install fonts-ipafont-gothic fonts-ipafont-mincho")
        print("!pip install japanize-matplotlib")
        # 英語ラベルを使用
        return False

    # フォント設定
    plt.rcParams['font.family'] = 'IPAexGothic, IPAGothic, sans-serif'
    plt.rcParams['font.sans-serif'] = ['IPAexGothic', 'IPAGothic', 'DejaVu Sans']
    plt.rcParams['axes.unicode_minus'] = False
    plt.rcParams['font.size'] = 14

    # Seabornの設定
    sns.set(style="whitegrid", font_scale=1.2)

    # 日本語フォントが使用可能
    return True

# ファイルパスを設定
file_path = "/content/drive/MyDrive/発表/2025日本眼科AI学会/patientlist'.xlsx"

def create_quartile_plots(file_path):
    # 日本語フォント設定
    use_japanese = setup_japanese_fonts()

    # Excelファイルを読み込む
    print(f"ファイル '{file_path}' を読み込んでいます...")
    df = pd.read_excel(file_path)

    # 元の分析と同様のフィルタリング
    print("データをフィルタリングしています...")
    df_filtered = df[(df['steroid pulse'] != 0) & (df['operation'] == '-')]

    # グループ分け
    group_negative_30_or_less = df_filtered[df_filtered['difference'] <= -30]
    group_other = df_filtered[df_filtered['difference'] > -30]

    print(f"difference ≤ -30群: {len(group_negative_30_or_less)}件")
    print(f"difference > -30群: {len(group_other)}件")

    # 右目と左目の値を分離する関数
    def split_and_convert_eye_values(group, column):
        right_eye, left_eye = [], []
        for value in group[column].dropna():
            if isinstance(value, str):
                parts = value.replace(' ', '').split(',')
                if len(parts) == 2:
                    try:
                        right_eye.append(float(parts[0]))
                        left_eye.append(float(parts[1]))
                    except ValueError:
                        # 数値変換に失敗した場合はスキップ
                        pass
        return pd.Series(right_eye), pd.Series(left_eye)

    # 眼関連の変数と非眼関連の変数を定義（元のコードから取得）
    eye_related_columns = [
        'MRD-1_pre', 'MRD1_post', 'MRD-2_pre', 'MRD2_post',
        'Hertel_pre', 'Hertel_post'
    ]
    non_eye_columns = ['Gorman score_pre', 'Gorman score_post']

    # 変数をグループ化して共通のスケールを設定するための辞書
    variable_groups = {
        'MRD1': ['MRD-1_pre', 'MRD1_post'],
        'MRD2': ['MRD-2_pre', 'MRD2_post'],
        'Hertel': ['Hertel_pre', 'Hertel_post'],
        'Gorman': ['Gorman score_pre', 'Gorman score_post']
    }

    # 日本語表示用の設定
    right_eye_label = "右眼 (R)" if use_japanese else "Right Eye (R)"
    left_eye_label = "左眼 (L)" if use_japanese else "Left Eye (L)"
    value_label = "値" if use_japanese else "Value"

    # 眼関連変数のための図を作成
    print("眼関連変数の4分位図を作成しています...")
    fig_eye, axes_eye = plt.subplots(len(eye_related_columns), 1,
                                     figsize=(12, 5*len(eye_related_columns)))

    # フォント設定の再適用（確実に適用するため）
    for prop in mpl.rcParams:
        if prop.startswith('font'):
            if prop in plt.rcParams:
                plt.rcParams[prop] = mpl.rcParams[prop]

    # 眼関連変数が1つの場合、axesは配列にならないため調整
    if len(eye_related_columns) == 1:
        axes_eye = [axes_eye]

    # 各メトリックタイプのスケール範囲を格納する辞書
    scale_ranges = {}

    # 各メトリックタイプごとにすべてのデータを収集し、スケール範囲を決定
    for group_name, columns in variable_groups.items():
        if group_name == 'Gorman':
            # Gormanスコアは1-4の範囲で固定
            scale_ranges[group_name] = (0.5, 4.5)
            continue

        all_values = []
        eye_columns = [col for col in columns if col in eye_related_columns]

        for column in eye_columns:
            # 全グループ、両眼のデータを収集
            for group in [group_negative_30_or_less, group_other]:
                right_eye, left_eye = split_and_convert_eye_values(group, column)
                all_values.extend(right_eye.values)
                all_values.extend(left_eye.values)

        if all_values:
            # 10%のマージンでスケール範囲を設定
            data_range = max(all_values) - min(all_values)
            margin = data_range * 0.1
            scale_ranges[group_name] = (min(all_values) - margin, max(all_values) + margin)

    # 各眼関連変数をプロット
    for i, column in enumerate(eye_related_columns):
        ax = axes_eye[i]

        # 両グループ、両眼のデータを取得
        right_eye_neg_30, left_eye_neg_30 = split_and_convert_eye_values(group_negative_30_or_less, column)
        right_eye_other, left_eye_other = split_and_convert_eye_values(group_other, column)

        # インデックスの問題を解決するため、新しいシリーズを作成
        right_eye_neg_30 = pd.Series(right_eye_neg_30.values)
        right_eye_other = pd.Series(right_eye_other.values)
        left_eye_neg_30 = pd.Series(left_eye_neg_30.values)
        left_eye_other = pd.Series(left_eye_other.values)

        # プロット用のDataFrameを作成（インデックスをリセット）
        plot_data = pd.DataFrame({
            'Value': pd.concat([right_eye_neg_30, right_eye_other, left_eye_neg_30, left_eye_other], ignore_index=True),
            'Group': pd.concat([
                pd.Series(['≤ -30'] * len(right_eye_neg_30)),
                pd.Series(['> -30'] * len(right_eye_other)),
                pd.Series(['≤ -30'] * len(left_eye_neg_30)),
                pd.Series(['> -30'] * len(left_eye_other))
            ], ignore_index=True),
            'Eye': pd.concat([
                pd.Series([right_eye_label] * len(right_eye_neg_30)),
                pd.Series([right_eye_label] * len(right_eye_other)),
                pd.Series([left_eye_label] * len(left_eye_neg_30)),
                pd.Series([left_eye_label] * len(left_eye_other))
            ], ignore_index=True)
        })

        # 箱ひげ図を作成（seaborn警告に対応）
        sns.boxplot(x='Eye', y='Value', hue='Group', data=plot_data,
                    palette=['#3498db', '#e74c3c'], ax=ax)

        # 個別データポイントを追加（seaborn警告に対応）
        sns.stripplot(x='Eye', y='Value', hue='Group', data=plot_data,
                     dodge=True, alpha=0.5, jitter=True, ax=ax,
                     palette=['#3498db', '#e74c3c'])

        # タイトルとラベルを設定
        ax.set_title(f'{column}', fontsize=14, fontweight='bold')
        ax.set_ylabel(value_label, fontsize=12)
        ax.set_xlabel('')

        # このメトリックのスケール範囲を設定
        for group_name, cols in variable_groups.items():
            if column in cols and group_name in scale_ranges:
                ax.set_ylim(scale_ranges[group_name])
                break

        # 凡例を設定（表示が重複するので最初の2つだけ表示）
        handles, labels = ax.get_legend_handles_labels()
        ax.legend(handles[:2], labels[:2], title='difference', loc='best')

        # サンプルサイズの注釈を追加
        ax.annotate(f'n(≤-30, R)={len(right_eye_neg_30)}, n(>-30, R)={len(right_eye_other)}\n'
                    f'n(≤-30, L)={len(left_eye_neg_30)}, n(>-30, L)={len(left_eye_other)}',
                    xy=(0.02, 0.02), xycoords='axes fraction', fontsize=10,
                    bbox=dict(boxstyle="round,pad=0.3", fc="white", ec="gray", alpha=0.8))

    plt.tight_layout()

    # 非眼関連変数のための図を作成
    print("非眼関連変数の4分位図を作成しています...")
    fig_non_eye, axes_non_eye = plt.subplots(len(non_eye_columns), 1,
                                           figsize=(10, 5*len(non_eye_columns)))

    # 非眼関連変数が1つの場合、axesは配列にならないため調整
    if len(non_eye_columns) == 1:
        axes_non_eye = [axes_non_eye]

    # 各非眼関連変数をプロット
    for i, column in enumerate(non_eye_columns):
        ax = axes_non_eye[i]

        # 両グループのデータを取得
        data_neg_30 = pd.to_numeric(group_negative_30_or_less[column], errors='coerce').dropna()
        data_other = pd.to_numeric(group_other[column], errors='coerce').dropna()

        # インデックスの問題を解決するため、新しいシリーズを作成
        data_neg_30 = pd.Series(data_neg_30.values)
        data_other = pd.Series(data_other.values)

        # プロット用のDataFrameを作成（インデックスをリセット）
        plot_data = pd.DataFrame({
            'Value': pd.concat([data_neg_30, data_other], ignore_index=True),
            'Group': pd.concat([
                pd.Series(['≤ -30'] * len(data_neg_30)),
                pd.Series(['> -30'] * len(data_other))
            ], ignore_index=True)
        })

        # 箱ひげ図を作成（seaborn警告に対応）
        sns.boxplot(x='Group', y='Value', hue='Group', data=plot_data,
                   palette=['#3498db', '#e74c3c'], ax=ax, legend=False)

        # 個別データポイントを追加（seaborn警告に対応）
        sns.stripplot(x='Group', y='Value', hue='Group', data=plot_data,
                     dodge=True, alpha=0.7, jitter=True, ax=ax,
                     palette=['#3498db', '#e74c3c'], legend=False)

        # タイトルとラベルを設定
        ax.set_title(f'{column}', fontsize=14, fontweight='bold')
        ax.set_ylabel(value_label, fontsize=12)
        ax.set_xlabel('difference', fontsize=12)

        # Gormanスコアのスケール設定（1-4）
        for group_name, cols in variable_groups.items():
            if column in cols and group_name in scale_ranges:
                ax.set_ylim(scale_ranges[group_name])

                # Gormanスコアの場合は整数の目盛りのみを表示
                if group_name == 'Gorman':
                    # 整数の目盛りのみを表示
                    import matplotlib.ticker as ticker
                    ax.yaxis.set_major_locator(ticker.MultipleLocator(1))
                    ax.set_yticks([1, 2, 3, 4])
                    ax.yaxis.set_major_formatter(ticker.FormatStrFormatter('%d'))
                break

        # サンプルサイズの注釈を追加
        ax.annotate(f'n(≤-30)={len(data_neg_30)}, n(>-30)={len(data_other)}',
                    xy=(0.02, 0.02), xycoords='axes fraction', fontsize=10,
                    bbox=dict(boxstyle="round,pad=0.3", fc="white", ec="gray", alpha=0.8))

    plt.tight_layout()

    # 画像のファイル名（英語で保存してエラー回避）
    eye_fig_filename = 'eye_variables_boxplot.png'
    non_eye_fig_filename = 'non_eye_variables_boxplot.png'

    # 図を保存
    print("図を保存しています...")
    fig_eye.savefig(eye_fig_filename, dpi=300, bbox_inches='tight')
    fig_non_eye.savefig(non_eye_fig_filename, dpi=300, bbox_inches='tight')

    print("完了しました！以下の2つの図が保存されました：")
    print(f"1. {eye_fig_filename}")
    print(f"2. {non_eye_fig_filename}")

    # 図を表示
    plt.show()

# ファイルパスを指定して関数を実行
if __name__ == "__main__":
    create_quartile_plots(file_path)